<a href="https://colab.research.google.com/github/micheleguidaa/robust-organ-scheduler/blob/main/robust_organ_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pianificazione Robusta dell'Orario di Prelievo in Interventi di Trapianto

**Studente:** Michele Guida  
**Corso:** Decision Support Systems  
**Repository:** [github.com/micheleguidaa/robust-organ-scheduler](https://github.com/micheleguidaa/robust-organ-scheduler)

In [1]:
# Import delle librerie necessarie
import numpy as np
import bisect
import random
from datetime import datetime, date, timedelta
from typing import Sequence, Mapping, Tuple, Optional, List, Dict

## 1. Definizione del Problema

Il progetto affronta la scelta dell'orario di inizio ($t_{start}$) di un intervento di prelievo multi-organo da un singolo donatore.
L'obiettivo è minimizzare i ritardi di partenza dei mezzi di trasporto assegnati ai diversi organi, tenendo conto
della diversa importanza clinica e dell'incertezza sulla durata dell'intervento.

### Modello Matematico

Sia $\mathbf{T}$ l'insieme discreto degli orari di inizio possibili e $\mathbf{D} = \{d_{min}, d_{med}, d_{max}\}$ l'insieme delle possibili durate dell'intervento.

Definiamo il **tempo di attesa** $\delta_i$ per l'organo $i$ come il tempo che intercorre tra la disponibilità dell'organo (fine intervento) e la prima partenza utile del mezzo di trasporto $S_i^j$:

$$
\delta_{i}(t_{start},d) = \min_{j: S_{i}^{j} \ge t_{start}+d} (S_{i}^{j} - (t_{start}+d))
$$

La **funzione di costo** (ritardo totale pesato) per uno scenario è data da:

$$
F(t_{start},d) = \sum_{i \in \mathbf{I}} w_{i} \cdot \delta_{i}(t_{start},d)
$$

### Obiettivo: Ottimizzazione Robusta (Minimax)
Poiché la durata $d$ è incerta, adottiamo un approccio robusto che minimizza il costo nel *caso peggiore* (worst-case scenario):

$$
t_{start}^* = \arg \min_{t_{start} \in \mathbf{T}} \left( \max_{d \in \mathbf{D}} F(t_{start}, d) \right)
$$

## 2. Soluzione e Implementazione dell'Algoritmo

Proponiamo due varianti dell'algoritmo in base alle assunzioni sulla struttura dei dati di input (orari di partenza degli organi).

### 2. Soluzione Unificata

Abbiamo implementato un'unica funzione `robust_organ_scheduler` che permette di scegliere l'algoritmo di ricerca più adatto in base alle caratteristiche dei dati di input, tramite il parametro `algorithm_type`.

#### Modalità: `unsorted` (Partenze NON Ordinate)
*   **Descrizione:** Non fa alcuna assunzione sull'ordinamento degli orari di partenza. Scansiona l'intera lista per trovare la prima partenza utile.
*   **Approccio:** Scansione Lineare (Linear Scan).
*   **Complessità:** $O\left( |\mathbf{T}| \cdot |\mathbf{D}| \cdot \sum_{i=1}^{N} |\mathbf{S}_i| \right)$.
*   **Utilizzo:** Adatta quando i dati provengono da fonti non strutturate.

#### Modalità: `sorted` (Partenze ORDINATE)
*   **Descrizione:** Assume che le liste degli orari di partenza $\mathbf{S}_i$ siano **già ordinate**. Utilizza la ricerca binaria per trovare efficientemente la partenza successiva.
*   **Approccio:** Ricerca Binaria (Binary Search).
*   **Complessità:** $O\left( |\mathbf{T}| \cdot |\mathbf{D}| \cdot \sum_{i=1}^{N} \log |\mathbf{S}_i| \right)$.
*   **Vantaggio:** Riduce drasticamente il tempo di calcolo per liste di partenze numerose.

In [2]:
# ---------------------------------------------------------
# 2. SOLUZIONE UNIFICATA (Sorted / Unsorted)
# ---------------------------------------------------------
def robust_organ_scheduler(
    orari_possibili_inizio_intervento: Sequence[datetime],
    durate_possibili_intervento: Sequence[timedelta],
    organi: Sequence[str],
    orari_partenza: Mapping[str, Sequence[datetime]],
    pesi: Mapping[str, float],
    algorithm_type: str = 'unsorted' # 'sorted' or 'unsorted'
) -> Tuple[float, Optional[datetime], timedelta, Dict[str, float]]:
    """
    Risolve il problema di pianificazione dell'orario di inizio intervento utilizzando un approccio di Ottimizzazione Robusta (Minimax).
    
    L'algoritmo cerca l'orario di inizio (t_start) che minimizza il massimo ritardo pesato (funzione di costo) 
    considerando tutti i possibili scenari di durata dell'intervento.
    
    Args:
        orari_possibili_inizio_intervento (Sequence[datetime]): Lista degli orari candidati per l'inizio dell'intervento.
        durate_possibili_intervento (Sequence[timedelta]): Insieme delle possibili durate dell'intervento (incertezza).
        organi (Sequence[str]): Lista degli organi da prelevare.
        orari_partenza (Mapping[str, Sequence[datetime]]): Dizionario che associa a ogni organo la lista dei suoi orari di partenza disponibili.
        pesi (Mapping[str, float]): Dizionario dei pesi (priorità clinica) per ciascun organo.
        algorithm_type (str, optional): Strategia di ricerca. 
            - 'sorted': Assume liste ordinate e usa Ricerca Binaria (O(log N)).
            - 'unsorted': Usa Scansione Lineare (O(N)). Default è 'unsorted'.

    Returns:
        Tuple[float, Optional[datetime], timedelta, Dict[str, float]]: Una tupla contenente:
            1. funzione_obiettivo (float): Il valore minimo del ritardo massimo pesato trovato (Minimax value).
            2. migliore_orario (Optional[datetime]): L'orario di inizio intervento che realizza il minimo. None se non trovato.
            3. tempo_esecuzione (timedelta): Tempo impiegato per il calcolo.
            4. migliori_attese (Dict[str, float]): Dizionario dei tempi di attesa (in minuti) per ciascun organo nella soluzione ottima.
    """
    inizio_esecuzione = datetime.now()
    funzione_obiettivo = np.inf
    migliore_orario = None
    migliori_attese = {}

    for orario_inizio in orari_possibili_inizio_intervento:
        somma_pesata_attese_max = -np.inf
        attese_worst_case_corrente = {}

        for durata in durate_possibili_intervento:
            somma_pesata_attese = 0.0
            orario_fine = orario_inizio + durata
            attese_correnti = {}

            for organo in organi:
                lista_partenze = orari_partenza[organo]
                attesa_minuti = np.inf
                
                if algorithm_type == 'sorted':
                    # RICERCA BINARIA (O(log N))
                    # bisect_left restituisce l'indice del primo elemento >= orario_fine
                    idx = bisect.bisect_left(lista_partenze, orario_fine)
                    
                    if idx < len(lista_partenze):
                        prima_partenza = lista_partenze[idx]
                        attesa_minuti = (prima_partenza - orario_fine).total_seconds() / 60
                        somma_pesata_attese += pesi[organo] * attesa_minuti
                    else:
                        somma_pesata_attese = np.inf
                        # Non facciamo break subito per poter salvare l'attesa infinita nel dizionario se necessario,
                        # ma per efficienza se la somma è inf possiamo fermarci dopo aver registrato.
                
                elif algorithm_type == 'unsorted':
                    # SCANSIONE LINEARE (O(N))
                    partenze_valide = [p for p in lista_partenze if p >= orario_fine]
                    
                    if partenze_valide:
                        prima_partenza = min(partenze_valide)
                        attesa_minuti = (prima_partenza - orario_fine).total_seconds() / 60
                        somma_pesata_attese += pesi[organo] * attesa_minuti
                    else:
                        somma_pesata_attese = np.inf
                else:
                    raise ValueError(f"Algorithm type '{algorithm_type}' not recognized. Use 'sorted' or 'unsorted'.")
                
                attese_correnti[organo] = attesa_minuti
                
                if somma_pesata_attese == np.inf:
                    break

            if somma_pesata_attese > somma_pesata_attese_max:
                somma_pesata_attese_max = somma_pesata_attese
                attese_worst_case_corrente = attese_correnti

        if somma_pesata_attese_max < funzione_obiettivo:
            funzione_obiettivo = somma_pesata_attese_max
            migliore_orario = orario_inizio
            migliori_attese = attese_worst_case_corrente

    tempo_esecuzione = datetime.now() - inizio_esecuzione
    return funzione_obiettivo, migliore_orario, tempo_esecuzione, migliori_attese

## 3. Verifica Funzionale (Test Case Semplice)

Prima di procedere con la sperimentazione su dati sintetici, eseguiamo un **Test Case Deterministico** per validare la correttezza dell'algoritmo.

In questo scenario semplificato:
*   Consideriamo solo **2 orari di inizio** candidati (08:00 e 09:00).
*   Assumiamo una **durata fissa** di 2 ore (nessuna incertezza sulla durata in questo test).
*   Gli orari di partenza dei mezzi sono stati costruiti ad-hoc per rendere l'orario delle **08:00** chiaramente preferibile rispetto alle 09:00.

L'obiettivo è confermare che la funzione `robust_organ_scheduler` identifichi correttamente l'ottimo globale (08:00) e calcoli il valore della funzione obiettivo atteso (5700).

In [ ]:
print("=== TEST CASE SEMPLICE (Verifica Manuale) ===")

# 1. Setup Scenario Semplificato
# Data base: Oggi alle 00:00
base_date = datetime.combine(date.today(), datetime.min.time())

# Due possibili orari di inizio: 08:00 e 09:00
test_orari_inizio = [
    base_date + timedelta(hours=8),
    base_date + timedelta(hours=9)
]

# Una sola durata possibile: 2 ore (per semplificare il calcolo)
test_durate = [timedelta(hours=2)]


# Organi richiesti
test_organi = ['Cuore', 'Polmoni', 'Fegato', 'Rene_sx', 'Rene_dx']

# Pesi specificati
test_pesi = {
    'Cuore': 100.0,
    'Polmoni': 90.0,
    'Fegato': 20.0,
    'Rene_sx': 20.0,
    'Rene_dx': 20.0
}

# Orari di partenza del trasporto
# Scenario costruito per favorire l'inizio alle 08:00.
# Se inizio 08:00 -> Fine 10:00.
# Se inizio 09:00 -> Fine 11:00.

test_partenze = {
    
    # Cuore (Peso 100): 
    # - 10:30 (attesa 30 min se fine 10:00)
    # - 12:00 (attesa 60 min se fine 11:00) -> Penalizza molto le 09:00
    
    'Cuore': [
        base_date + timedelta(hours=10, minutes=30), 
        base_date + timedelta(hours=12, minutes=0)
    ],
    
    # Polmoni (Peso 90):
    # - 10:30 (attesa 30 min se fine 10:00)
    # - 11:30 (attesa 30 min se fine 11:00) -> Neutro
    
    'Polmoni': [
        base_date + timedelta(hours=10, minutes=30), 
        base_date + timedelta(hours=11, minutes=30)
    ],
    
    # Altri organi (Peso 20): partenze comode per entrambi gli orari
    'Fegato': [base_date + timedelta(hours=10, minutes=0), base_date + timedelta(hours=11, minutes=0)], # Attesa 0
    'Rene_sx': [base_date + timedelta(hours=10, minutes=0), base_date + timedelta(hours=11, minutes=0)], # Attesa 0
    'Rene_dx': [base_date + timedelta(hours=10, minutes=0), base_date + timedelta(hours=11, minutes=0)]  # Attesa 0
}

print(f"Orari Inizio Candidati: {[t.strftime('%H:%M') for t in test_orari_inizio]}")
print(f"Durata Intervento: {test_durate[0]}")
print("Pesi:", test_pesi)

# 2. Esecuzione
val_obj, best_time, _, best_waits = robust_organ_scheduler(
    test_orari_inizio, 
    test_durate, 
    test_organi, 
    test_partenze, 
    test_pesi,
    algorithm_type='sorted'
)

# 3. Verifica
# Calcolo Costi Previsti:

# Caso A (Inizio 08:00 -> Fine 10:00):
# - Cuore: parte 10:30 -> attesa 30 min * 100 = 3000
# - Polmoni: parte 10:30 -> attesa 30 min * 90 = 2700
# - Altri: partono 10:00 -> attesa 0
# Totale A = 5700

# Caso B (Inizio 09:00 -> Fine 11:00):
# - Cuore: parte 12:00 -> attesa 60 min * 100 = 6000
# - Polmoni: parte 11:30 -> attesa 30 min * 90 = 2700
# - Altri: partono 11:00 -> attesa 0
# Totale B = 8700

# L'algoritmo deve scegliere 08:00.

print(f"\n--- Risultato Atteso ---")
print("Miglior orario: 08:00")
print("Costo atteso (approx): 5700")

print(f"\n--- Risultato Ottenuto ---")
print(f"Miglior orario: {best_time.strftime('%H:%M') if best_time else 'None'}")
print(f"Valore Obiettivo: {val_obj}")
print("Attese calcolate:", best_waits)

assert best_time == test_orari_inizio[0], "Errore: L'orario scelto non è quello atteso!"
assert val_obj == 5700.0, f"Errore: Il costo calcolato ({val_obj}) non è 5700!"
print("\n>>> TEST SUPERATO CON SUCCESSO! <<<")

=== TEST CASE SEMPLICE (Verifica Manuale) ===
Orari Inizio Candidati: ['08:00', '09:00']
Durata Intervento: 2:00:00
Pesi: {'Cuore': 100.0, 'Polmoni': 90.0, 'Fegato': 20.0, 'Rene_sx': 20.0, 'Rene_dx': 20.0}

--- Risultato Atteso ---
Miglior orario: 08:00
Costo atteso (approx): 5700

--- Risultato Ottenuto ---
Miglior orario: 08:00
Valore Obiettivo: 5700.0
Attese calcolate: {'Cuore': 30.0, 'Polmoni': 30.0, 'Fegato': 0.0, 'Rene_sx': 0.0, 'Rene_dx': 0.0}

>>> TEST SUPERATO CON SUCCESSO! <<<


# Test e Valutazione delle Prestazioni

## Obiettivo
La fase di test ha lo scopo di valutare le prestazioni delle due varianti dell’algoritmo utilizzando dataset sintetici con caratteristiche controllate.  

---

## Pesi degli Organi

Nell’esperimento sono stati assegnati pesi differenziati agli organi per riflettere la loro diversa priorità clinica.

$$
w_{\text{Cuore}} = 100,\quad
w_{\text{Polmoni}} = 90,\quad
w_{\text{Rene\,Sx}} = 20,\quad
w_{\text{Rene\,Dx}} = 20,\quad
w_{\text{Fegato}} = 20.
$$

Questi pesi vengono utilizzati nella funzione obiettivo per modulare il contributo del tempo di attesa dei singoli organi.

---

## Generazione dei Dati Sintetici

### 1. Orari di inizio intervento

Gli orari di inizio vengono generati come una sequenza crescente avente queste regole:

- `t_0 = 0`
- `t_1 ∈ [0.5 h ,1 h, 1.5 h, 2 h]`
- `t_2 ∈ (t_1 , t_1 + 0.5 h, ..., 3 h]`
- `t_3 ∈ (t_2, t_2 + 0.5 h, ..., 5 h]`

Da notare come gli orari vengano generati con passi di 30 minuti (0.5 h).

Tale sequenza soddisfa:

$$
0 = t_0 < t_1 < t_2 < t_3 \le 5\text{ h}.
$$

L’insieme dei tempi ammissibili è:

$$
\mathbf{T} = \{t_0,\; t_1,\; t_2,\; t_3\}.
$$

---

### 2. Orari di partenza degli organi

Per ogni organo $ i $, gli orari di partenza sono generati come valori uniformi nell’intervallo discreto:

$$
S_i^j \in [0,\; 0.5,\; 1,\; \ldots,\; 23,\; 23.5\;] \;\text{h}
$$

Anche in questo caso, gli orari sono generati con passi di 30 minuti (0.5 h).

Ciascun organo randomicamente riceve un numero di orari di partenza compreso tra 10 e 35.

---

### 3. Durata dell’intervento

La durata dell’intervento è considerata incerta e può assumere uno dei seguenti valori:

$$
d \in \{2,\; 4,\; 6\}\text{ h}.
$$

---

## Protocollo di Test

Per ogni configurazione sperimentale:

- l’esperimento viene ripetuto **50 volte** con diversi dataset generati casualmente;
- per ciascun run vengono registrati e salvati:
  - i parametri di input generati,
  - il tempo di esecuzione,
  - la soluzione ottimale,
  - il tempo di attesa di ciascun organo.

Dai risultati si ricavano:

- il **tempo medio e deviazione standard di esecuzione**,  
- il **valore medio e deviazione standard de lla soluzione pesata**,  
- il **tempo medio di attesa per organo**:
- il **tempo massimo di attesa per organo**:

---

## Analisi di Sensitività sulla Densità degli Orari di Partenza

Per valutare l’effetto della riduzione della densità degli orari di partenza:

1. si ordinano i valori $ S_i^j $ per ogni organo;
2. si rimuove un orario ogni due (pattern alternato);
3. si ripete l’intero processo di test sulla nuova configurazione.

L’analisi consente di esaminare l’impatto della variazione della granularità dei dati su:

- tempo di esecuzione,
- qualità della soluzione,
- robustezza dell’algoritmo (in presenza dei pesi $ w_i $).

---


In [ ]:
# ---------------------------------------------------------
# CONFIGURAZIONE SPERIMENTALE
# ---------------------------------------------------------
ORGANI = ['Cuore', 'Polmoni', 'Rene_sx', 'Rene_dx', 'Fegato']

# Pesi aggiornati come da descrizione nel testo
PESO_ORGANI = {
    'Cuore': 100.0,
    'Polmoni': 90.0,
    'Rene_sx': 20.0,     
    'Rene_dx': 20.0,
    'Fegato': 20.0
}

DURATA_POSSIBILI_INTERVENTO = [
    timedelta(hours=2),
    timedelta(hours=4),
    timedelta(hours=6),
]

RUN = 50

# ---------------------------------------------------------
# 1. GENERAZIONE ORARI POSSIBILI DI INIZIO INTERVENTO
# ---------------------------------------------------------
def genera_orari_possibili_inizio_intervento(
    inizio_giorno: datetime = datetime.combine(date.today(), datetime.min.time())
) -> Sequence[datetime]:
    """
    Genera 4 istanti temporali t0, t1, t2, t3 per l'inizio dell'intervento.
    t0 = 0
    t1 in [0.5, 1, 1.5, 2] ore
    t2 in (t1, 3] ore
    t3 in (t2, 5] ore
    """
    
    passo = 30 # minuti
    
    # t_0 = 0
    t0_min = 0

    # t_1: scelta tra 30, 60, 90, 120 minuti
    possibili_t1 = [30, 60, 90, 120]
    t1_min = random.choice(possibili_t1)

    # t_2: strettamente maggiore di t1, fino a 180 min (3h)
    # range(start, stop, step) -> start = t1 + 30
    possibili_t2 = list(range(t1_min + passo, 180 + 1, passo))
    t2_min = random.choice(possibili_t2)

    # t_3: strettamente maggiore di t2, fino a 300 min (5h)
    possibili_t3 = list(range(t2_min + passo, 300 + 1, passo))
    t3_min = random.choice(possibili_t3)

    # Creazione lista datetime
    orari = [
        inizio_giorno + timedelta(minutes=t0_min),
        inizio_giorno + timedelta(minutes=t1_min),
        inizio_giorno + timedelta(minutes=t2_min),
        inizio_giorno + timedelta(minutes=t3_min)
    ]
    
    return orari

# ---------------------------------------------------------
# 2. GENERAZIONE ORARI DI PARTENZA DEGLI ORGANI
# ---------------------------------------------------------
def genera_orari_partenza_organi(
    organi: List[str],
    inizio_giorno: datetime = datetime.combine(date.today(), datetime.min.time())
) -> Dict[str, List[datetime]]:
    """
    Genera gli orari di partenza per una lista di organi.
    Dominio: [0, 0.5, ..., 24h]
    Numero orari per organo: Random tra 10 e 35.
    """
    
    # Definizione del dominio temporale [0, 24h] a passi di 30 min
    # 24 ore * 60 min = 1440 min. 
    # Usiamo +1 nel range per includere l'estremo superiore (1440 min = 24:00).
    pool_minuti = list(range(0, (24 * 60) + 1, 30))
    
    risultato = {}

    for organo in organi:
        # Scelta numero di orari (tra 10 e 35)
        n_scelte = random.randint(10, 35)
        
        # Campionamento casuale senza ripetizioni
        scelte_minuti = random.sample(pool_minuti, n_scelte)
        
        # Ordinamento cronologico
        scelte_minuti.sort()
        
        # Conversione in datetime
        lista_orari = []
        for m in scelte_minuti:
            orario = inizio_giorno + timedelta(minutes=m)
            lista_orari.append(orario)
            
        risultato[organo] = lista_orari

    return risultato

In [ ]:
# ---------------------------------------------------------
#  VERIFICA GENERAZIONE DATI (Sanity Check)
# ---------------------------------------------------------
print("=== TEST 1: Orari Inizio Intervento ===",end='\n')
print("\nT = [", end=' ')
orari_intervento = genera_orari_possibili_inizio_intervento()

for i, orario in enumerate(orari_intervento):
    print(f"t_{i}: {orario.strftime('%H:%M')}", end=' ')
print("]")

print("\n=== TEST 2: Orari Partenza Organi ===")
orari_organi = genera_orari_partenza_organi(ORGANI)

for organo, orari in orari_organi.items():
    # Calcoliamo quante partenze ci sono
    num_partenze = len(orari)
    # Formattiamo per la stampa
    str_orari = [o.strftime("%H:%M") for o in orari]
    
    print(f"\n--- {organo} (Peso: {PESO_ORGANI.get(organo, 'N/A')}) ---")
    print(f"Totale partenze: {num_partenze}")
    print(f"Primi 3 orari: {str_orari[:3]}")
    print(f"Ultimi 3 orari: {str_orari[-3:]}")

=== TEST 1: Orari Inizio Intervento ===

T = [ t_0: 00:00 t_1: 01:30 t_2: 03:00 t_3: 03:30 ]

=== TEST 2: Orari Partenza Organi ===

--- Cuore (Peso: 100.0) ---
Totale partenze: 33
Primi 3 orari: ['00:00', '00:30', '01:30']
Ultimi 3 orari: ['22:30', '23:00', '00:00']

--- Polmoni (Peso: 90.0) ---
Totale partenze: 22
Primi 3 orari: ['00:00', '01:30', '02:30']
Ultimi 3 orari: ['22:00', '23:00', '00:00']

--- Rene_sx (Peso: 20.0) ---
Totale partenze: 23
Primi 3 orari: ['00:30', '01:00', '02:00']
Ultimi 3 orari: ['23:00', '23:30', '00:00']

--- Rene_dx (Peso: 20.0) ---
Totale partenze: 32
Primi 3 orari: ['01:30', '02:30', '03:30']
Ultimi 3 orari: ['22:30', '23:00', '00:00']

--- Fegato (Peso: 20.0) ---
Totale partenze: 12
Primi 3 orari: ['00:30', '02:00', '03:30']
Ultimi 3 orari: ['16:00', '16:30', '21:00']


In [ ]:
# ---------------------------------------------------------
# ESEMPIO DI SINGOLA ESECUZIONE (Single Run)
# ---------------------------------------------------------
print("=== ESEMPIO DI SINGOLA ESECUZIONE ===")

# 1. Generazione Dati
orari_intervento = genera_orari_possibili_inizio_intervento()
orari_partenze = genera_orari_partenza_organi(ORGANI)

print(f"Orari intervento possibili: {[t.strftime('%H:%M') for t in orari_intervento]}")
print(f"Durate possibili intervento: {[str(d) for d in DURATA_POSSIBILI_INTERVENTO]}")

print("\nOrari di Partenza Generati (Lista Completa):")
for organo, lista in orari_partenze.items():
    lista_str = [t.strftime('%H:%M') for t in lista]
    print(f"- {organo} ({len(lista)} partenze): {lista_str}")

# 2. Esecuzione Algoritmo
obj_val, opt_time, exec_time, wait_times = robust_organ_scheduler(
    orari_intervento,
    DURATA_POSSIBILI_INTERVENTO,
    ORGANI,
    orari_partenze,
    PESO_ORGANI,
    algorithm_type='sorted'
)

# 3. Stampa Risultati
print(f"\nRisultati Ottimizzazione:")
print(f"Miglior orario inizio intervento: {opt_time.strftime('%H:%M') if opt_time else 'Nessuna soluzione'}")
print(f"Valore Funzione Obiettivo: {obj_val:.2f}")
print(f"Tempo di esecuzione: {exec_time.total_seconds() * 1000:.4f} ms")

print("\nDettaglio Attese per Organo nella soluzione ottima:")
if wait_times:
    for organo, attesa in wait_times.items():
        print(f"- {organo}: {attesa:.2f} min")
else:
    print("Nessuna attesa calcolata (soluzione non trovata).")

=== ESEMPIO DI SINGOLA ESECUZIONE ===
Orari intervento possibili: ['00:00', '02:00', '02:30', '03:30']
Durate possibili intervento: ['2:00:00', '4:00:00', '6:00:00']

Orari di Partenza Generati (Lista Completa):
- Cuore (25 partenze): ['00:00', '00:30', '01:30', '02:00', '03:00', '03:30', '04:30', '06:00', '07:30', '08:30', '09:00', '10:00', '11:00', '12:00', '13:00', '13:30', '17:30', '18:30', '19:30', '20:00', '20:30', '22:00', '23:00', '23:30', '00:00']
- Polmoni (31 partenze): ['00:00', '00:30', '02:00', '02:30', '03:00', '03:30', '04:00', '05:30', '06:00', '07:00', '08:00', '10:30', '11:00', '12:00', '13:30', '15:00', '16:00', '16:30', '17:00', '17:30', '18:00', '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:30', '23:30', '00:00']
- Rene_sx (33 partenze): ['00:00', '00:30', '01:00', '02:30', '03:00', '03:30', '05:00', '05:30', '06:00', '08:00', '08:30', '09:30', '10:00', '11:00', '11:30', '12:00', '12:30', '13:30', '14:00', '14:30', '15:00', '15:30', '16:30', '

In [ ]:
# ---------------------------------------------------------
# FASE SPERIMENTALE
# ---------------------------------------------------------
def battery_test(organi, peso_organi, durate_possibili_intervento, num_run, riduci_densita=False):
    
    tempi_esecuzione = []
    valori_obiettivo = []
    attese_per_organo = {o: [] for o in organi}

    print(f"\n\nAvvio test batteria ({num_run} run) - Modalità riduci_densita={riduci_densita}...")

    for i in range(num_run):
        # Generazione orari inizio intervento
        orari_intervento = genera_orari_possibili_inizio_intervento()
        
        # Generazione orari partenza organi
        orari_partenze_organi = genera_orari_partenza_organi(organi)

        if riduci_densita:
            # Rimuove un orario ogni due (pattern alternato)
            for organo in organi:
                # Assumiamo che siano già ordinati dalla funzione di generazione
                lista = orari_partenze_organi[organo]
                # Slice [::2] prende elementi a indici 0, 2, 4...
                orari_partenze_organi[organo] = lista[::2]
        
        # Esecuzione algoritmo robusto (usiamo 'sorted' come default per efficienza)
        valore_funzione_obiettivo, orario_ottimo, tempo_esecuzione, attese_ottimali = robust_organ_scheduler(
            orari_intervento, 
            durate_possibili_intervento, 
            organi, 
            orari_partenze_organi, 
            peso_organi,
            algorithm_type='sorted'
        )
        
        # Registrazione metriche
        tempi_esecuzione.append(tempo_esecuzione.total_seconds() * 1000) # in ms
        valori_obiettivo.append(valore_funzione_obiettivo)

        # Registrazione attese per lo scenario peggiore corrispondente alla soluzione ottima
        if orario_ottimo is not None:
            for organo, attesa in attese_ottimali.items():
                attese_per_organo[organo].append(attesa)

    # Calcolo statistiche
    mean_exec = np.mean(tempi_esecuzione)
    std_exec = np.std(tempi_esecuzione)
    
    mean_obj = np.mean(valori_obiettivo)
    std_obj = np.std(valori_obiettivo)

    print("\n--- RISULTATI ---")
    print(f"Tempo medio esecuzione: {mean_exec:.4f} ms (std: {std_exec:.4f})")
    print(f"Valore medio funzione obiettivo: {mean_obj:.2f} (std: {std_obj:.2f})")
    
    print("\nStatistiche per Organo (Attesa Worst-Case):")
    print(f"{'Organo':<10} | {'Media (minuti)':<12} | {'Max (minuti)':<10}")
    print("-" * 36)
    for o in organi:
        if attese_per_organo[o]:
            # Filtriamo eventuali infiniti per le statistiche
            valid_attese = [x for x in attese_per_organo[o] if x != np.inf]
            if valid_attese:
                m = np.mean(valid_attese)
                mx = np.max(valid_attese)
                print(f"{o:<10} | {m:<12.2f} | {mx:<10.2f}")
            else:
                print(f"{o:<10} | {'Inf':<12} | {'Inf':<10}")
        else:
             print(f"{o:<10} | {'N/A':<12} | {'N/A':<10}")
    print("-" * 36)

In [ ]:
# Esecuzione Test Standard
battery_test(ORGANI, PESO_ORGANI, DURATA_POSSIBILI_INTERVENTO, RUN, riduci_densita=False)



Avvio test batteria (50 run) - Modalità riduci_densita=False...

--- RISULTATI ---
Tempo medio esecuzione: 0.0257 ms (std: 0.0020)
Valore medio funzione obiettivo: 12720.00 (std: 8179.17)

Statistiche per Organo (Attesa Worst-Case):
Organo     | Media (minuti) | Max (minuti)
------------------------------------
Cuore      | 42.60        | 390.00    
Polmoni    | 55.20        | 300.00    
Rene_sx    | 48.00        | 300.00    
Rene_dx    | 56.40        | 240.00    
Fegato     | 70.20        | 420.00    
------------------------------------


In [ ]:
# Esecuzione Test Analisi di Sensitività (Densità Ridotta)
battery_test(ORGANI, PESO_ORGANI, DURATA_POSSIBILI_INTERVENTO, RUN, riduci_densita=True)



Avvio test batteria (50 run) - Modalità riduci_densita=True...

--- RISULTATI ---
Tempo medio esecuzione: 0.0258 ms (std: 0.0020)
Valore medio funzione obiettivo: 22272.00 (std: 12574.91)

Statistiche per Organo (Attesa Worst-Case):
Organo     | Media (minuti) | Max (minuti)
------------------------------------
Cuore      | 95.40        | 450.00    
Polmoni    | 93.60        | 450.00    
Rene_sx    | 70.80        | 420.00    
Rene_dx    | 74.40        | 240.00    
Fegato     | 70.20        | 540.00    
------------------------------------
